In [6]:
import matplotlib.pyplot as plt # for plotting
import numpy as np #array handling 
import scipy as sp #scientific package
import pandas as pd #csv file handling
from scipy.optimize import * #for optimizing the function
from scipy.optimize import fsolve
import time

In [7]:
def alpha(x,y,iavg): 
    lx = len(x) 
    a = [] 
    t = [] 
    for k in range(lx-iavg):
        yd = y[k:k+iavg] 
        xd = x[k:k+iavg] 
        popt,pcov = np.polyfit(xd,yd,deg=1)
        t.append(np.mean(xd))
        a.append(popt)
        
    return 10**np.asarray(t),np.asarray(a)

def haranu(rt,D,del_1,tdl1,del_2,tdl2):
    return 6*D*rt + C*(del_1*(1-np.exp(-(rt/tdl1))) + del_2*(1-np.exp(-(rt/tdl2))))

def petrov(rt,D,del_1,tdl1,del_2,tdl2):
    H = haranu(rt=rt,D=D,del_1=del_1,tdl1=tdl1,del_2=del_2,tdl2=tdl2)
    return 1/((1 + (2/3.0)*H/(width))* np.sqrt(1+(2/3.0)*H/(depth)))

def nalpha(rt,D,del_1,tdl1,del_2,tdl2):
    rt_dl1 = np.exp(-(rt/tdl1))
    rt_dl2 = np.exp(-(rt/tdl2))
    return rt*(6*D + C*(del_1*rt_dl1/tdl1 + del_2*rt_dl2/tdl2 ))/(6*D*rt + C*(del_1*(1-rt_dl1) + del_2*(1-rt_dl2)))

In [8]:
## The below function is create a set of 180 curves, with moving window of ws i.e. 30 curves (15 mins)
subs = 220
star = 20
n_2_3 = 2.0/3.0
def runner(df,tcurve,slr):
    
    fda = np.zeros((5,tcurve))
    
    tcur = []
    acur = []
    
    p0 = [1.37874036e-11, 2.7e+02, 9.06751393e-06, 4.6e+02, 1.16446007e-04]

    ltc =len(list(df))
    
    for j in range(tcurve):

        X1 = np.random.randint(low=1, high=ltc, size=(slr,))
        ll = df.iloc[:,X1]
        avgn = ll.mean(axis=1)
        yf = avgn/np.mean(avgn[4:20])
        
        try :
            popt,pcov = curve_fit(petrov,xf[10:],yf[10:],p0=p0,maxfev = 10000)
            fda[:,j] = popt
        except:
            pass
        

        sol = fsolve(lambda MSD: 1/((1 + n_2_3*MSD/width)*np.sqrt(1+n_2_3*MSD/depth)) - [yf[i] for i in range(star,lxf-subs)], yf[star:lxf-subs]*1e-14 )
        xsol = np.log10(xf[star:lxf-subs][sol>0])
        sol = np.log10(sol[sol>0])
        
        t,a = alpha(xsol,sol,66)
        
        tcur.append(t)
        acur.append(a)  

    return np.asarray(fda),np.asarray(tcur),np.asarray(acur)

In [10]:
KbT    = (1.38e-23)*298
etta   = 8.9e-04
pi     = 3.14
C      = KbT/etta
a      = 1.0e-09
L      = 222e-09 
Llp    = 2.0
itr    = 6
gama   = 0.5772
mu0    = 1.0/(6*pi*etta*a)
amu0   = a*mu0
SP     = 6.7
width  = 4*71.66*284e-06*1e-12
depth  = SP*SP*width
st     = 60
en     = 280
tcurve = 10
preavg = 20
df     = pd.read_csv(r'GTP_EEA1.csv')
xf     = df['t']
nhr    = int(1.5*120)
lxf    = len(xf)

In [11]:
fda   = np.zeros((3,tcurve))

name   = [r'GTP_EEA1.csv']

slr = 20
for j in range(len(name)):
    start  = time.perf_counter()

    df      = pd.read_csv(name[j],encoding='utf-7')
    
    emb,emt,ema     = runner(df,tcurve,preavg)

    naam    = name[j][:-4]+'_'+str(tcurve)+'_'+str(slr)+'cal_L220_fit'
    np.save(naam,emb)
    
    naam    = name[j][:-4]+'_'+str(tcurve)+'_'+str(slr)+'cal_L220_tcur'
    np.save(naam,emt)
    
    naam    = name[j][:-4]+'_'+str(tcurve)+'_'+str(slr)+'cal_L220_acur'
    np.save(naam,ema)
    
    finish  = time.perf_counter()
    print(f'finished in {round(finish-start,2)} second(s)')

C:\Users\anupam\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\anupam\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\anupam\AppData\Local\Temp\ipykernel_22404\3862076632.py:30: RuntimeWarning: invalid value encountered in sqrt
  sol = fsolve(lambda MSD: 1/((1 + n_2_3*MSD/width)*np.sqrt(1+n_2_3*MSD/depth)) - [yf[i] for i in range(star,lxf-subs)], yf[star:lxf-subs]*1e-14 )
C:\Users\anupam\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


finished in 21.97 second(s)


C:\Users\anupam\AppData\Local\Temp\ipykernel_22404\3862076632.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(fda),np.asarray(tcur),np.asarray(acur)
